### Introduction: This is the Jupyter notebook to do the flowing things:

1. Read slimmed PKU Tree files
2. Store the pre-fit shapes histograms to pickle files

kernel:HWW



### Import necessary modules

In [1]:
import numpy as np
import os
import pandas as pd
import random
import awkward as ak
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
import boost_histogram as bh
from scipy import interpolate
# from sklearn.metrics import roc_curve, auc
from cycler import cycler
import uproot
# means uproot4
import ROOT
import argparse
import glob
import json
import logging
import os
import pickle as pkl
import warnings

import hist as hist2
import pyarrow
# import utils #local file: utils.py
import yaml
from typing import Dict, List, Union
from dataclasses import dataclass
from copy import deepcopy

# from coffea import hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods import vector
# from coffea.nanoevents.methods.vector import PtEtaPhiMLorentzVector

/data/pku/home/zhaoyz/anaconda3/envs/HWW/etc/cling/std.modulemap:257:29: warning: unknown attribute 'optional' [-Wignored-attributes]
  module "memory_resource" [optional] {
                            ^
/data/pku/home/zhaoyz/anaconda3/envs/HWW/etc/cling/std.modulemap:447:27: warning: unknown attribute 'optional' [-Wignored-attributes]
  module "bits/chrono.h" [optional] {
                          ^
/data/pku/home/zhaoyz/anaconda3/envs/HWW/etc/cling/std.modulemap:531:41: warning: unknown attribute 'optional' [-Wignored-attributes]
  explicit module "bits_ranges_base_h" [optional] {
                                        ^
/data/pku/home/zhaoyz/anaconda3/envs/HWW/etc/cling/std.modulemap:538:32: warning: unknown attribute 'optional' [-Wignored-attributes]
  module "bits/ranges_util.h" [optional] {
                               ^
/data/pku/home/zhaoyz/anaconda3/envs/HWW/etc/cling/std.modulemap:559:40: warning: unknown attribute 'optional' [-Wignored-attributes]
  module "bits/uses_allo

Welcome to JupyROOT 6.28/04


### Read SlimmedTree files

In [2]:
MODE = "templates"
# year = "2016"
# year = "2017"
# year = "2018"
year = "Full-Run2"
if MODE == "slimmedtree_validation":
    CustNanoData = {
        'data'        : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Data/SlimmedTree_Data.root"%(year),
        'QCD'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/MC/SlimmedTree_QCD.root"%(year),          
        'WJets'       : "/data/bond/zhaoyz/SlimmedTree/V5/%s/MC/SlimmedTree_WJets.root"%(year),
        'Rest'        : "/data/bond/zhaoyz/SlimmedTree/V5/%s/ttbar_validation_final/Slimmed_Rest_ST.root"%(year),
        'TotalSignal' : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_Total.root"%(year),
        'ggF'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_GluGlu.root"%(year),
        'VH'          : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_VH.root"%(year),
        'ttH'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_ttH.root"%(year),
        'VBF'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_VBF.root"%(year),
        'ttbar_val'   : "/data/bond/zhaoyz/SlimmedTree/V5/%s/ttbar_validation_final/Slimmed_ttbar_validation.root"%(year),
    }
elif MODE == "tree_validation":
        CustNanoData = {
        'data'        : "/data/bond/zhaoyz/Tree/V8/%s/Merged/Data/Tree_Data.root"%(year),
        'QCD'         : "/data/bond/zhaoyz/Tree/V8/%s/Merged/MC/Tree_QCD.root"%(year),          
        'WJets'       : "/data/bond/zhaoyz/Tree/V8/%s/Merged/MC/Tree_WJets.root"%(year),
        'Rest'        : "/data/bond/zhaoyz/Tree/V8/%s/Merged/ttbar_validation_final/Rest_ST.root"%(year),
        'TotalSignal' : "/data/bond/zhaoyz/Tree/V8/%s/Merged/Signal/Tree_Total.root"%(year),
        'ggF'         : "/data/bond/zhaoyz/Tree/V8/%s/Merged/Signal/Tree_GluGlu.root"%(year),
        'VH'          : "/data/bond/zhaoyz/Tree/V8/%s/Merged/Signal/Tree_VH.root"%(year),
        'ttH'         : "/data/bond/zhaoyz/Tree/V8/%s/Merged/Signal/Tree_ttH.root"%(year),
        'VBF'         : "/data/bond/zhaoyz/Tree/V8/%s/Merged/Signal/Tree_VBF.root"%(year),
        'ttbar_val'   : "/data/bond/zhaoyz/Tree/V8/%s/Merged/ttbar_validation_final/ttbar_validation.root"%(year),
    }
else:
    CustNanoData = {
        'data'        : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Data/SlimmedTree_Data.root"%(year),
        'QCD'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/MC/SlimmedTree_QCD.root"%(year),
        'Top'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/MC/SlimmedTree_Top.root"%(year),
        'WJets'       : "/data/bond/zhaoyz/SlimmedTree/V5/%s/MC/SlimmedTree_WJets.root"%(year),
        'Rest'        : "/data/bond/zhaoyz/SlimmedTree/V5/%s/MC/SlimmedTree_Rest.root"%(year),
        'TotalSignal' : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_Total.root"%(year),
        'ggF'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_GluGlu.root"%(year),
        'VH'          : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_VH.root"%(year),
        'ttH'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_ttH.root"%(year),
        'VBF'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_VBF.root"%(year),
    }    
BKG = ["QCD","Top","WJets","Rest"]
files = {typefile : {} for typefile in CustNanoData}
for typefile in CustNanoData:
    files[typefile] = uproot.lazy({CustNanoData[typefile]: "PKUTree" })

### Get DPhi in the events

In [3]:
def get_dphi(events):
    pT_higgs   = events["PTj_V2_a"]
    eta_higgs  = events["Etaj_V2_a"]
    phi_higgs  = events["Phij_V2_a"]
    mass_higgs = events["Mj_V2_a"]
    pT_MET = events["MET_et"]
    eta_MET = events["Etaj_V2_a"]
    phi_MET = events["MET_phi"]
    mass_MET = ak.zeros_like(events["MET_phi"])
    vec_higgs = ak.zip({
        "pt"   : pT_higgs   ,
        "eta"  : eta_higgs  ,
        "phi"  : phi_higgs  ,
        "mass" : mass_higgs ,
    },
    with_name="PtEtaPhiMLorentzVector",
    behavior=vector.behavior,
    )
    vec_MET = ak.zip({
        "pt"   :   pT_MET   ,
        "eta"  :  eta_MET  ,
        "phi"  :  phi_MET  ,
        "mass" : mass_MET ,
    },
    with_name="PtEtaPhiMLorentzVector",
    behavior=vector.behavior,
    )
    delta_phi = np.subtract(vec_MET.phi, vec_higgs.phi)
    delta_phi = np.where(delta_phi > np.pi, delta_phi - 2*np.pi, delta_phi)
    delta_phi = np.where(delta_phi < -np.pi, delta_phi + 2*np.pi, delta_phi)
    delta_phi = np.abs(delta_phi)
    print(delta_phi)
    events["DPhi"] = delta_phi

for k in files:
    print("Add dphi of:",k)
    get_dphi(events=files[k])

Add dphi of: data


[0.597, 0.363, 0.458, 1.07, 2.84, 0.341, ... 0.767, 0.2, 0.504, 3.03, 0.289, 0.0353]
Add dphi of: QCD
[2.8, 2.58, 0.514, 2.3, 3.01, 0.305, 0.629, ... 0.0396, 3.07, 0.765, 3, 2.96, 3.07]
Add dphi of: Top
[0.377, 0.707, 0.49, 2.48, 2.35, 2.75, 2.95, ... 0.169, 2.6, 2.94, 1.61, 1.35, 3.05]
Add dphi of: WJets
[2.81, 0.123, 1.6, 0.0544, 2.76, 2.51, ... 2.74, 0.508, 2.42, 1.41, 2.56, 0.258]
Add dphi of: Rest
[2.56, 1.73, 0.401, 3.13, 3.03, 2.83, 1.66, ... 1.26, 1.24, 0.0949, 2.46, 1.87, 2.91]
Add dphi of: TotalSignal
[2.4, 1.87, 3.06, 3.04, 2.91, 1.51, 2.08, ... 3.1, 0.141, 0.273, 0.498, 1.18, 2.73]
Add dphi of: ggF
[2.4, 1.87, 3.06, 3.04, 2.91, 1.51, 2.08, ... 3.09, 0.722, 3, 3.11, 3.07, 0.455]
Add dphi of: VH
[0.168, 1.39, 2.76, 1.26, 0.0118, 2.83, ... 3.1, 0.141, 0.273, 0.498, 1.18, 2.73]
Add dphi of: ttH
[3.12, 2.62, 2.28, 2.03, 0.0168, 0.16, ... 0.743, 0.839, 2.31, 2.44, 2.99, 0.506]
Add dphi of: VBF
[2.97, 2.31, 0.0849, 0.17, 0.0194, 0.481, ... 1.13, 0.332, 2.5, 2.82, 2.01, 2.45]


In [4]:

def get_reco(events):
    pT_higgs   = events["PTj_V2_a"]
    eta_higgs  = events["Etaj_V2_a"]
    phi_higgs  = events["Phij_V2_a"]
    mass_higgs = events["Mj_V2_a"]
    pT_MET = events["MET_et"]
    eta_MET = events["Etaj_V2_a"]
    phi_MET = events["MET_phi"]
    mass_MET = ak.zeros_like(events["MET_phi"])
    vec_higgs = ak.zip({
        "pt"   : pT_higgs   ,
        "eta"  : eta_higgs  ,
        "phi"  : phi_higgs  ,
        "mass" : mass_higgs ,
    },
    with_name="PtEtaPhiMLorentzVector",
    behavior=vector.behavior,
    )

    vec_MET = ak.zip({
        "pt"   :   pT_MET   ,
        "eta"  :  eta_MET  ,
        "phi"  :  phi_MET  ,
        "mass" : mass_MET ,
    },
    with_name="PtEtaPhiMLorentzVector",
    behavior=vector.behavior,
    )

    vec_sum = vec_MET + vec_higgs
    mH_reco = vec_sum.mass
    events["MH_Reco"] = ak.where(( (events["DPhi"] < 0.8) & (events["MET_et"]/events["PTj_V2_a"] > 0.2)),mH_reco, events["Mj_V2_a"])
get_reco(files["QCD"]) 
print("done QCD")
get_reco(files["data"]) 
print("done data")  
get_reco(files["ttbar_val"]) if MODE == "validation" else get_reco(files["Top"])
get_reco(files["WJets"])
get_reco(files["Rest"])
# get_reco(files["WplusRest"])
print("done bkg") 
get_reco(files["TotalSignal"])
get_reco(files["ggF"])
get_reco(files["VH"])
get_reco(files["ttH"])
get_reco(files["VBF"])
print("done signal") 

done QCD
done data
done bkg
done signal


### some test about variables

In [5]:
files["VBF"]["DPhi"]

<Array [2.97, 2.31, 0.0849, ... 2.01, 2.45] type='17017 * float32'>

### plot setting

In [6]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
import boost_histogram as bh
from cycler import cycler

use_helvet = False ## true: use helvetica for plots, make sure the system have the font installed
if use_helvet:
    CMShelvet = hep.style.CMS
    CMShelvet['font.sans-serif'] = ['Helvetica', 'Arial']
    plt.style.use(CMShelvet)
else:
    plt.style.use(hep.style.CMS)

def flow(hist: bh.Histogram, overflow: bool=False, underflow: bool=False):
    h, var = hist.view(flow=(overflow | underflow)).value, hist.view(flow=(overflow | underflow)).variance
    if overflow: 
        # h, var also include underflow bins but in plots usually no underflow data
        # And we've filled None with -999, so we shouldn't show underflow data (mostly from filled None)
        # You have to access the overflow and underflow bins data like below:
        h[-2] += h[-1]; var[-2] += var[-1]
    if underflow:
        h[1] += h[0]; var[1] += var[0]
    if overflow or underflow:
        h, var = h[1:-1], var[1:-1]
    return h, var
    # Return the updated histogram and variance

def error_bar(h, var, type='data'):
    from scipy.interpolate import CubicSpline
    if type == 'data':
        number = h
    elif type == 'mc':  # h = k*N, var = k^2*N, std = k*sqrt(N)
        number = h**2 / var
    else:
        raise ValueError("type should be 'data' or 'mc'! ")
    center = range(11) # Number: 0-10
    up = np.array([1.84, 3.30, 4.64, 5.92, 7.16, 8.38, 9.58, 10.77, 11.95, 13.11, 14.27]) - center
    down = center - np.array([0, 0.17, 0.71, 1.37, 2.09, 2.84, 3.62, 4.42, 5.23, 6.06, 6.89])
    #cs means to create a CubicSpline object
    cs_up = CubicSpline(x=center, y=up)
    cs_down = CubicSpline(x=center, y=down)
    
    Garwood = (number>0)&(number<10)
    poison_error_bar = np.sqrt(number)
    up_error_bar = np.copy(poison_error_bar)
    down_error_bar = np.copy(poison_error_bar)
    up_error_bar[Garwood] = cs_up(number[Garwood])
    down_error_bar[Garwood] = cs_down(number[Garwood])
    if type == 'mc':
        up_error_bar *= var/h
        down_error_bar *= var/h
    up_error_bar [up_error_bar < 0 ] = 0
    down_error_bar [down_error_bar < 0 ] = 0
    return np.array([down_error_bar, up_error_bar])


# function to find the optimal region with S/sqrt(B)
# not used so far
def optimalcut(shist, bhist):
    n_bins = len(shist)
    best_lower = None
    best_upper = None
    best_s_sqrt_b = 0

    for lower in range(n_bins):
        for upper in range(lower+1, n_bins+1):
            s = np.sum(shist[lower:upper])
            b = np.sum(bhist[lower:upper])
            s_sqrt_b = s / np.sqrt(b + 1)

            if s_sqrt_b > best_s_sqrt_b:
                best_lower = lower
                best_upper = upper
                best_s_sqrt_b = s_sqrt_b

    return best_lower, best_upper, best_s_sqrt_b

### Define observable object variables

In [7]:
@dataclass
class ShapeVar:
    """Class to store attributes of a variable to make a histogram of.

    Args:
        var (str): variable name
        label (str): variable label
        bins (List[int]): bins
        reg (bool, optional): Use a regular axis or variable binning. Defaults to True.
        blind_window (List[int], optional): if blinding, set min and max values to set 0. Defaults to None.
        significance_dir (str, optional): if plotting significance, which direction to plot it in.
          See more in plotting.py:ratioHistPlot(). Options are ["left", "right", "bin"]. Defaults to "right".
    """

    var: str = None
    label: str = None
    bins: List[int] = None
    reg: bool = True #regular axis
    blind_window: List[int] = None
    significance_dir: str = "right"

    def __post_init__(self):
        # create axis used for histogramming
        if self.reg:
            self.axis = hist2.axis.Regular(*self.bins, name=self.var, label=self.label)
        else:
            self.axis = hist2.axis.Variable(self.bins, name=self.var, label=self.label)
def blindBins(h: hist2.Hist, blind_region: List, blind_sample: str = None, axis=0):
    """
    Blind (i.e. zero) bins in histogram ``h``.
    If ``blind_sample`` specified, only blind that sample, else blinds all.
    """
    if axis > 0:
        raise Exception("not implemented > 1D blinding yet")

    bins = h.axes[axis + 1].edges
    lv = int(np.searchsorted(bins, blind_region[0], "right"))
    rv = int(np.searchsorted(bins, blind_region[1], "left") + 1)

    if blind_sample is not None:
        data_key_index = np.where(np.array(list(h.axes[0])) == blind_sample)[0][0]
        h.view(flow=True)[data_key_index][lv:rv].value = 0
        h.view(flow=True)[data_key_index][lv:rv].variance = 0
    else:
        h.view(flow=True)[:, lv:rv].value = 0
        h.view(flow=True)[:, lv:rv].variance = 0       
shape_vars = [
    ShapeVar(
        "MH_Reco",
        r"Higgs candidate MET recovery mass [GeV]",
        [20, 40, 240],
        reg=True,
        blind_window=[80, 160],
    )
]


### Define CUT(aka. regions)

In [10]:
CUT = {        
        "SR1a" : {k: ( (files[k]["a_HWW_V2"] > 0.975) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] < 0.3)) for k in files},
        "SR1b" : {k: ( (files[k]["a_HWW_V2"] > 0.8) & (files[k]["a_HWW_V2"] <= 0.975) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] < 0.3)) for k in files}, 
        "SR2a" : {k: ( (files[k]["DPhi"] < 0.8)     & (files[k]["a_HWW_V2"] > 0.975)  & (files[k]["MET_et"]/files[k]["PTj_V2_a"] >= 0.3) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] < 0.6)) for k in files},
        "SR2b" : {k: ( (files[k]["a_HWW_V2"] > 0.8) & (files[k]["a_HWW_V2"] <= 0.975) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] >= 0.3) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] < 0.6)) for k in files},
        "SR3a" : {k: ( (files[k]["DPhi"] < 0.8) & (files[k]["a_HWW_V2"] > 0.975) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] >= 0.6)) for k in files},
        "SR3b" : {k: ( (files[k]["DPhi"] < 0.8) & (files[k]["a_HWW_V2"] <= 0.975) & (files[k]["a_HWW_V2"] > 0.8) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] >= 0.6)) for k in files},
        "CR1"  : {k: ( (files[k]["a_HWW_V2"] > 0.6) & (files[k]["a_HWW_V2"] <= 0.8) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] < 0.3)) for k in files}, 
        "CR2"  : {k: ( (files[k]["a_HWW_V2"] > 0.6) & (files[k]["a_HWW_V2"] <= 0.8) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] >= 0.3) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] < 0.6)) for k in files},       
        "CR3"  : {k: ( (files[k]["DPhi"] < 0.8) & (files[k]["a_HWW_V2"] <= 0.8) & (files[k]["a_HWW_V2"] > 0.6) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] >= 0.6)) for k in files},
    }

# CR: 0.6-0.8

In [11]:
CUT_BLINDED = {
        "SR1a_blinded" : {k: ( ((files[k]["MH_Reco"] <= 80) | (files[k]["MH_Reco"] >= 160)) & (files[k]["a_HWW_V2"] > 0.975) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] < 0.3)) for k in files},
        "SR1b_blinded" : {k: ( ((files[k]["MH_Reco"] <= 80) | (files[k]["MH_Reco"] >= 160)) & (files[k]["a_HWW_V2"] > 0.8) & (files[k]["a_HWW_V2"] <= 0.975) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] < 0.3)) for k in files}, 
        "SR2a_blinded" : {k: ( ((files[k]["MH_Reco"] <= 80) | (files[k]["MH_Reco"] >= 160)) & (files[k]["DPhi"] < 0.8)     & (files[k]["a_HWW_V2"] > 0.975)  & (files[k]["MET_et"]/files[k]["PTj_V2_a"] >= 0.3) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] < 0.6)) for k in files},
        "SR2b_blinded" : {k: ( ((files[k]["MH_Reco"] <= 80) | (files[k]["MH_Reco"] >= 160)) & (files[k]["a_HWW_V2"] > 0.8) & (files[k]["a_HWW_V2"] <= 0.975) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] >= 0.3) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] < 0.6)) for k in files},
        "SR3a_blinded" : {k: ( ((files[k]["MH_Reco"] <= 80) | (files[k]["MH_Reco"] >= 160)) & (files[k]["DPhi"] < 0.8) & (files[k]["a_HWW_V2"] > 0.975) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] >= 0.6)) for k in files},
        "SR3b_blinded" : {k: ( ((files[k]["MH_Reco"] <= 80) | (files[k]["MH_Reco"] >= 160)) & (files[k]["DPhi"] < 0.8) & (files[k]["a_HWW_V2"] <= 0.975) & (files[k]["a_HWW_V2"] > 0.8) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] >= 0.6)) for k in files},
        "CR1_blinded"  : {k: ( ((files[k]["MH_Reco"] <= 80) | (files[k]["MH_Reco"] >= 160)) & (files[k]["a_HWW_V2"] > 0.6) & (files[k]["a_HWW_V2"] <= 0.8) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] < 0.3)) for k in files}, 
        "CR2_blinded"  : {k: ( ((files[k]["MH_Reco"] <= 80) | (files[k]["MH_Reco"] >= 160)) & (files[k]["a_HWW_V2"] > 0.6) & (files[k]["a_HWW_V2"] <= 0.8) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] >= 0.3) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] < 0.6)) for k in files},       
        "CR3_blinded"  : {k: ( ((files[k]["MH_Reco"] <= 80) | (files[k]["MH_Reco"] >= 160)) & (files[k]["DPhi"] < 0.8) & (files[k]["a_HWW_V2"] <= 0.8) & (files[k]["a_HWW_V2"] > 0.6) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] >= 0.6)) for k in files},
}

### Save hist templates to pkl files

We note here that no particular operation is needed for QCD, since we only need raw QCD MC ratio as initial tranfer factor in the actual QCD prediction, and rhalphabet method will use (data - other bkg) in fail(control) region and perform simultaneous fit with pass(signal) region

In [8]:

def save_pkl(files, template_file = "./templates/19Jan2024/hists_templates_run2.pkl"):
    regions = ["SR1a","SR1b","CR1","SR2a","SR2b","CR2","SR3a","SR3b","CR3"] #signal regions or control regions
    # regions = ["SR1a","SR1b","CR1a","CR1b","SR2a","SR2b","CR2a","CR2b","SR3a","SR3b","CR3a","CR3b"]
    # samples = list(['data','QCD','Top','WJets','Rest','TotalSignal','ggF','VH','ttH','VBF']) 
    samples = list(['data','QCD','Top','WJets','Rest','ggF','VH','ttH','VBF']) # ignore the combined "TotalSignal" for a test
    weights = [] #uncertainties to be added 
    templates = {}
    for region in regions:
        templates[region] = hist2.Hist(
            hist2.axis.StrCategory(samples, name="Sample"),
            # hist2.axis.StrCategory([], name="systematic", growth=True),
            *[shape_var.axis for shape_var in shape_vars],
            storage="weight",
            ) #initialize a hist object
        for sample in samples:
            data = files[sample][CUT[region][sample]] if ("blinded" not in region) else files[sample][CUT_BLINDED[region][sample]]
            templates[region].fill(
                Sample=sample,
                # systematic="nominal",
                MH_Reco=data["MH_Reco"],
                weight=data["weight"],
            )
            if sample == "data": 
                if (region.endswith("a") or region.endswith("b")):
                    # blind signal mass windows in pass region in data
                    print("blind data of ",region)
                    for i, shape_var in enumerate(shape_vars):
                        if shape_var.blind_window is not None:
                            blindBins(templates[region], shape_var.blind_window, "data", axis=i) 
            for weight in weights:
                pass #uncertainties to be added
        print("done fill template ",region)        
    #Creates blinded copies of each region's templates and saves a pickle of the templates
    blind_window = shape_vars[0].blind_window
    for label, template in list(templates.items()):
        blinded_template = deepcopy(template)
        blindBins(blinded_template, blind_window)
        templates[f"{label}Blinded"] = blinded_template
    with open(template_file, "wb") as fp:
        pkl.dump(templates, fp) # dump the templates of each region in a pkl file
        print("Saved templates to", template_file)

In [12]:
save_pkl(files = files)


blind data of  SR1a
done fill template  SR1a
blind data of  SR1b
done fill template  SR1b
done fill template  CR1
blind data of  SR2a
done fill template  SR2a
blind data of  SR2b
done fill template  SR2b
done fill template  CR2
blind data of  SR3a
done fill template  SR3a
blind data of  SR3b
done fill template  SR3b
done fill template  CR3
Saved templates to ./templates/19Jan2024//hists_templates_run2.pkl


### some test about the output templates

In [13]:
with open("/home/pku/zhaoyz/Higgs/boostedHWW/combine/templates/19Jan2024/hists_templates_run2.pkl","rb") as f:
    hists_template1 = pkl.load(f)
# hists_template["pass"]["QCD",:]  
# hists_template["pass"]["QCD",:].sum().value
# hists_template["CR2"]["QCD",:]
hists_template1["SR1aBlinded"]["data",:]

Hist(Regular(20, 40, 240, name='MH_Reco', label='Higgs candidate MET recovery mass [GeV]'), storage=Weight()) # Sum: WeightedSum(value=33745, variance=33745) (WeightedSum(value=37202, variance=37202) with flow)

In [14]:
hists_template1["SR1aBlinded"]

Hist(
  StrCategory(['data', 'QCD', 'Top', 'WJets', 'Rest', 'ggF', 'VH', 'ttH', 'VBF'], name='Sample'),
  Regular(20, 40, 240, name='MH_Reco', label='Higgs candidate MET recovery mass [GeV]'),
  storage=Weight()) # Sum: WeightedSum(value=71255.2, variance=226989) (WeightedSum(value=78256, variance=237184) with flow)

In [18]:
sample_template1 = hists_template1["SR1a"]["QCD", :]
err = sample_template1.variances()
err

array([ 8548.042071  , 26988.6981184 , 25601.98032062, 27327.66276296,
       26665.95630566, 27548.07397173, 34296.01006913, 31758.43711341,
       34311.02309932, 27169.786575  , 31183.93480812, 24178.97432329,
       20200.20107508, 21832.23205841, 18085.3307364 , 15020.97656324,
        9943.52714521,  9197.25103296,  5388.71222649,  4594.94066619])

In [20]:
for i , axis in enumerate(hists_template1["SR1a"].axes[1:]):
    print(i, axis)

0 Regular(20, 40, 240, name='MH_Reco', label='Higgs candidate MET recovery mass [GeV]')


### Some test of HHbbVV analysis for reference

In [21]:
with open("/home/pku/zhaoyz/Higgs/HHbbVV/src/HHbbVV/postprocessing/templates/23Jun14/2018_templates.pkl","rb") as f:
    hists_template2 = pkl.load(f)
# hists_template["pass"]["QCD",:]  
# hists_template["pass"]["QCD",:].sum().value


In [22]:
hists_template2.keys()

dict_keys(['pass', 'fail', 'pass_JES_up', 'fail_JES_up', 'pass_JES_down', 'fail_JES_down', 'pass_JER_up', 'fail_JER_up', 'pass_JER_down', 'fail_JER_down', 'pass_JMS_up', 'fail_JMS_up', 'pass_JMS_down', 'fail_JMS_down', 'pass_JMR_up', 'fail_JMR_up', 'pass_JMR_down', 'fail_JMR_down', 'passBlinded', 'failBlinded', 'pass_JES_upBlinded', 'fail_JES_upBlinded', 'pass_JES_downBlinded', 'fail_JES_downBlinded', 'pass_JER_upBlinded', 'fail_JER_upBlinded', 'pass_JER_downBlinded', 'fail_JER_downBlinded', 'pass_JMS_upBlinded', 'fail_JMS_upBlinded', 'pass_JMS_downBlinded', 'fail_JMS_downBlinded', 'pass_JMR_upBlinded', 'fail_JMR_upBlinded', 'pass_JMR_downBlinded', 'fail_JMR_downBlinded'])

In [23]:
hists_template2["passBlinded"]

Hist(
  StrCategory(['HHbbVV', 'ggHH_kl_2p45_kt_1_HHbbVV', 'ggHH_kl_5_kt_1_HHbbVV', 'ggHH_kl_0_kt_1_HHbbVV', 'qqHH_CV_1_C2V_1_kl_1_HHbbVV', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV', 'qqHH_CV_0p5_C2V_1_kl_1_HHbbVV', 'QCD', 'TT', 'ST', 'V+Jets', 'Diboson', 'Data', 'HHbbVV_txbb_down', 'ggHH_kl_2p45_kt_1_HHbbVV_txbb_down', 'ggHH_kl_5_kt_1_HHbbVV_txbb_down', 'ggHH_kl_0_kt_1_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_1_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_txbb_down', 'qqHH_CV_0p5_C2V_1_kl_1_HHbbVV_txbb_down', 'HHbbVV_pileup_down', 'ggHH_kl_2p45_kt_1_HHbbVV_pileup_down', 'ggHH_kl_5_kt_1_HHbbVV_pileup_down', 'ggHH_kl_0_kt_1_HHbbVV_pileup_down', 'qqHH_CV_1_C2V_1_kl_1_HHbbVV_pileup_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_pileup_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_pileup_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_pileup_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_pileup_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_pileup_down', 'qqHH_CV_0p5_C2V_1_kl_1_HHbbVV_pileup_down', 'QCD_pileup_down', 'TT_pileup_down', 'ST_pileup_down', 'V+Jets_pileup_down', 'Diboson_pileup_down', 'HHbbVV_ISRPartonShower_down', 'ggHH_kl_2p45_kt_1_HHbbVV_ISRPartonShower_down', 'ggHH_kl_5_kt_1_HHbbVV_ISRPartonShower_down', 'ggHH_kl_0_kt_1_HHbbVV_ISRPartonShower_down', 'V+Jets_ISRPartonShower_down', 'HHbbVV_FSRPartonShower_down', 'ggHH_kl_2p45_kt_1_HHbbVV_FSRPartonShower_down', 'ggHH_kl_5_kt_1_HHbbVV_FSRPartonShower_down', 'ggHH_kl_0_kt_1_HHbbVV_FSRPartonShower_down', 'V+Jets_FSRPartonShower_down', 'HHbbVV_L1EcalPrefiring_down', 'ggHH_kl_2p45_kt_1_HHbbVV_L1EcalPrefiring_down', 'ggHH_kl_5_kt_1_HHbbVV_L1EcalPrefiring_down', 'ggHH_kl_0_kt_1_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1_C2V_1_kl_1_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_0p5_C2V_1_kl_1_HHbbVV_L1EcalPrefiring_down', 'QCD_L1EcalPrefiring_down', 'TT_L1EcalPrefiring_down', 'ST_L1EcalPrefiring_down', 'V+Jets_L1EcalPrefiring_down', 'Diboson_L1EcalPrefiring_down', 'HHbbVV_txbb_up', 'ggHH_kl_2p45_kt_1_HHbbVV_txbb_up', 'ggHH_kl_5_kt_1_HHbbVV_txbb_up', 'ggHH_kl_0_kt_1_HHbbVV_txbb_up', 'qqHH_CV_1_C2V_1_kl_1_HHbbVV_txbb_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_txbb_up', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_txbb_up', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_txbb_up', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_txbb_up', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_txbb_up', 'qqHH_CV_0p5_C2V_1_kl_1_HHbbVV_txbb_up', 'HHbbVV_pileup_up', 'ggHH_kl_2p45_kt_1_HHbbVV_pileup_up', 'ggHH_kl_5_kt_1_HHbbVV_pileup_up', 'ggHH_kl_0_kt_1_HHbbVV_pileup_up', 'qqHH_CV_1_C2V_1_kl_1_HHbbVV_pileup_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_pileup_up', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_pileup_up', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_pileup_up', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_pileup_up', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_pileup_up', 'qqHH_CV_0p5_C2V_1_kl_1_HHbbVV_pileup_up', 'QCD_pileup_up', 'TT_pileup_up', 'ST_pileup_up', 'V+Jets_pileup_up', 'Diboson_pileup_up', 'HHbbVV_ISRPartonShower_up', 'ggHH_kl_2p45_kt_1_HHbbVV_ISRPartonShower_up', 'ggHH_kl_5_kt_1_HHbbVV_ISRPartonShower_up', 'ggHH_kl_0_kt_1_HHbbVV_ISRPartonShower_up', 'V+Jets_ISRPartonShower_up', 'HHbbVV_FSRPartonShower_up', 'ggHH_kl_2p45_kt_1_HHbbVV_FSRPartonShower_up', 'ggHH_kl_5_kt_1_HHbbVV_FSRPartonShower_up', 'ggHH_kl_0_kt_1_HHbbVV_FSRPartonShower_up', 'V+Jets_FSRPartonShower_up', 'HHbbVV_L1EcalPrefiring_up', 'ggHH_kl_2p45_kt_1_HHbbVV_L1EcalPrefiring_up', 'ggHH_kl_5_kt_1_HHbbVV_L1EcalPrefiring_up', 'ggHH_kl_0_kt_1_HHbbVV_L1EcalPrefiring_up', 'qqHH_CV_1_C2V_1_kl_1_HHbbVV_L1EcalPrefiring_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_L1EcalPrefiring_up', 'qqHH_CV_1p5_C2

In [24]:
hists_template2["passBlinded"]["Data",:]

Hist(Regular(20, 50, 250, name='bbFatJetParticleNetMass', label='$m^{bb}_{Reg}$ (GeV)'), storage=Weight()) # Sum: WeightedSum(value=30, variance=30)

In [25]:
hists_template2["passBlinded"]

Hist(
  StrCategory(['HHbbVV', 'ggHH_kl_2p45_kt_1_HHbbVV', 'ggHH_kl_5_kt_1_HHbbVV', 'ggHH_kl_0_kt_1_HHbbVV', 'qqHH_CV_1_C2V_1_kl_1_HHbbVV', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV', 'qqHH_CV_0p5_C2V_1_kl_1_HHbbVV', 'QCD', 'TT', 'ST', 'V+Jets', 'Diboson', 'Data', 'HHbbVV_txbb_down', 'ggHH_kl_2p45_kt_1_HHbbVV_txbb_down', 'ggHH_kl_5_kt_1_HHbbVV_txbb_down', 'ggHH_kl_0_kt_1_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_1_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_txbb_down', 'qqHH_CV_0p5_C2V_1_kl_1_HHbbVV_txbb_down', 'HHbbVV_pileup_down', 'ggHH_kl_2p45_kt_1_HHbbVV_pileup_down', 'ggHH_kl_5_kt_1_HHbbVV_pileup_down', 'ggHH_kl_0_kt_1_HHbbVV_pileup_down', 'qqHH_CV_1_C2V_1_kl_1_HHbbVV_pileup_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_pileup_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_pileup_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_pileup_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_pileup_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_pileup_down', 'qqHH_CV_0p5_C2V_1_kl_1_HHbbVV_pileup_down', 'QCD_pileup_down', 'TT_pileup_down', 'ST_pileup_down', 'V+Jets_pileup_down', 'Diboson_pileup_down', 'HHbbVV_ISRPartonShower_down', 'ggHH_kl_2p45_kt_1_HHbbVV_ISRPartonShower_down', 'ggHH_kl_5_kt_1_HHbbVV_ISRPartonShower_down', 'ggHH_kl_0_kt_1_HHbbVV_ISRPartonShower_down', 'V+Jets_ISRPartonShower_down', 'HHbbVV_FSRPartonShower_down', 'ggHH_kl_2p45_kt_1_HHbbVV_FSRPartonShower_down', 'ggHH_kl_5_kt_1_HHbbVV_FSRPartonShower_down', 'ggHH_kl_0_kt_1_HHbbVV_FSRPartonShower_down', 'V+Jets_FSRPartonShower_down', 'HHbbVV_L1EcalPrefiring_down', 'ggHH_kl_2p45_kt_1_HHbbVV_L1EcalPrefiring_down', 'ggHH_kl_5_kt_1_HHbbVV_L1EcalPrefiring_down', 'ggHH_kl_0_kt_1_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1_C2V_1_kl_1_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_L1EcalPrefiring_down', 'qqHH_CV_0p5_C2V_1_kl_1_HHbbVV_L1EcalPrefiring_down', 'QCD_L1EcalPrefiring_down', 'TT_L1EcalPrefiring_down', 'ST_L1EcalPrefiring_down', 'V+Jets_L1EcalPrefiring_down', 'Diboson_L1EcalPrefiring_down', 'HHbbVV_txbb_up', 'ggHH_kl_2p45_kt_1_HHbbVV_txbb_up', 'ggHH_kl_5_kt_1_HHbbVV_txbb_up', 'ggHH_kl_0_kt_1_HHbbVV_txbb_up', 'qqHH_CV_1_C2V_1_kl_1_HHbbVV_txbb_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_txbb_up', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_txbb_up', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_txbb_up', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_txbb_up', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_txbb_up', 'qqHH_CV_0p5_C2V_1_kl_1_HHbbVV_txbb_up', 'HHbbVV_pileup_up', 'ggHH_kl_2p45_kt_1_HHbbVV_pileup_up', 'ggHH_kl_5_kt_1_HHbbVV_pileup_up', 'ggHH_kl_0_kt_1_HHbbVV_pileup_up', 'qqHH_CV_1_C2V_1_kl_1_HHbbVV_pileup_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_pileup_up', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_pileup_up', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_pileup_up', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_pileup_up', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_pileup_up', 'qqHH_CV_0p5_C2V_1_kl_1_HHbbVV_pileup_up', 'QCD_pileup_up', 'TT_pileup_up', 'ST_pileup_up', 'V+Jets_pileup_up', 'Diboson_pileup_up', 'HHbbVV_ISRPartonShower_up', 'ggHH_kl_2p45_kt_1_HHbbVV_ISRPartonShower_up', 'ggHH_kl_5_kt_1_HHbbVV_ISRPartonShower_up', 'ggHH_kl_0_kt_1_HHbbVV_ISRPartonShower_up', 'V+Jets_ISRPartonShower_up', 'HHbbVV_FSRPartonShower_up', 'ggHH_kl_2p45_kt_1_HHbbVV_FSRPartonShower_up', 'ggHH_kl_5_kt_1_HHbbVV_FSRPartonShower_up', 'ggHH_kl_0_kt_1_HHbbVV_FSRPartonShower_up', 'V+Jets_FSRPartonShower_up', 'HHbbVV_L1EcalPrefiring_up', 'ggHH_kl_2p45_kt_1_HHbbVV_L1EcalPrefiring_up', 'ggHH_kl_5_kt_1_HHbbVV_L1EcalPrefiring_up', 'ggHH_kl_0_kt_1_HHbbVV_L1EcalPrefiring_up', 'qqHH_CV_1_C2V_1_kl_1_HHbbVV_L1EcalPrefiring_up', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_L1EcalPrefiring_up', 'qqHH_CV_1p5_C2

### test how to load and use the *.pkl template file

In [ ]:
def get_template(h, sample):
    ''' 
    histogram h Hist, with axes:["samples","systematic","MH_Reco"]
    sample is sample name in ["QCD",...,"data"]
    '''
    mass_axis = 1 #axis index
    massbins = h.axes[mass_axis].edges
    return (h[sample, :].values(), massbins, "MH_Reco")

a = get_template(hists_templates1["SR3a"],"QCD")
a

(array([ 0.        ,  2.61734887,  2.40244829, 13.38662184,  0.        ,
        12.94938391,  7.15780488, 11.0105996 , 18.07070539,  0.        ,
         7.82469779,  0.89133593,  1.93871653,  2.80038463,  3.449436  ,
         0.53863144,  0.2711149 ,  0.3287236 ,  0.        ,  2.0529086 ]),
 array([ 40.,  50.,  60.,  70.,  80.,  90., 100., 110., 120., 130., 140.,
        150., 160., 170., 180., 190., 200., 210., 220., 230., 240.]),
 'MH_Reco')

### Some other test

In [ ]:
regions = {
        "CR1" :{"SRa": "SR1a","SRb":"SR1b"},
        "CR2" :{"SRa": "SR2a","SRb":"SR2b"},
        "CR3" :{"SRa": "SR3a","SRb":"SR3b"},
        }

regions_blinded = { key_fail + "_blinded": {key_pass + "_blinded" : key_pass_ab + "_blinded" for key_pass , key_pass_ab in key_pass_dict.items()}  for key_fail , key_pass_dict in regions.items()}
regions_blinded.keys()

dict_keys(['CR1_blinded', 'CR2_blinded', 'CR3_blinded'])

In [ ]:
region = "SR1a_blinded"
pass_region = ("a_" in region)
pass_region

True

In [ ]:
region = "SR1aBlinded"
region_noblinded = region.split("Blinded")[0]
region_noblinded

'SR1a'